Plan-and-Execute with LastMile Auto Instrumentation

This notebook demonstrates the concept of a "Plan-and-Execute" LangChain agent with LastMile auto instrumentation. The agent accomplishes an objective by first planning what to do and then executing the sub-tasks. LastMile auto instrumentation is used to trace and monitor the execution of the agent.

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain_experimental
!pip install langchain_openai
!pip install wikipedia
!pip install numexpr
!pip install lastmile_eval

In [22]:
# Imports

from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

# LastMile Auto Instrumentation Setup

An instance of `LangChainInstrumentor` is created with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [23]:
from lastmile_eval.rag.debugger.tracing.auto_instrumentation import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
instrumentor = LangChainInstrumentor(project_name="Plan-and-Execute Example")
instrumentor.instrument()

2024-05-17 14:36:29,871 - Overriding of current TracerProvider is not allowed
2024-05-17 14:36:29,873 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-17 14:36:30,006 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=Plan-and-Execute+Example HTTP/1.1" 200 359
2024-05-17 14:36:30,048 - DependencyConflict: requested: "langchain_core >= 0.1.0" but found: "packaging 24.0"


# Tools

The tools that the agent will use for executing sub-tasks are defined. In this example, we have two tools:
- "Wikipedia" tool: Uses the `WikipediaAPIWrapper` to look up information on Wikipedia.
- "Calculator" tool: Uses the `LLMMathChain` to perform mathematical calculations.

In [26]:
# Define the tools that the agent will use for executing sub-tasks

wikipedia = WikipediaAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up information on Wikipedia",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to perform mathematical calculations",
    ),
]


2024-05-17 14:39:48,095 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-17 14:39:48,096 - load_verify_locations cafile='/Users/ankush/anaconda3/envs/empty2/lib/python3.11/site-packages/certifi/cacert.pem'
2024-05-17 14:39:48,105 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-17 14:39:48,106 - load_verify_locations cafile='/Users/ankush/anaconda3/envs/empty2/lib/python3.11/site-packages/certifi/cacert.pem'


# Run Example

The agent is run with a sample question that involves looking up the population of the capital city of the country where the Eiffel Tower is located and calculating the square root of that number. The result is printed.

In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

# ## Run Example

# Run the agent with a sample question
question = "What is the population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."
result = agent.run(question)

In [18]:
print(result)

The square root of the population of Paris is approximately 1450.05.




## Imports

The necessary libraries are imported, including `langchain` for the agent and tools, `langchain_openai` for the language models, and `lastmile_eval` for auto instrumentation.




## Planner, Executor, and Agent

The planner, executor, and agent are set up using the `load_chat_planner`, `load_agent_executor`, and `PlanAndExecute` classes, respectively. The planner and executor use a `ChatOpenAI` model, and the executor is configured with the defined tools.
